2022 - 5/6

In [56]:
import os
import re
import json
import glob
import string
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import time
from datetime import timedelta

# Time Check

In [24]:
Export = '/Users/wenlanzhang/Downloads/PhD_UCL/Data/Twitter/Nairobi_1922/Time/'

## Time dictionary

In [ ]:
Weekday = [1, 2, 3, 4, 5]
Weekend = [0,6]

# WeekDay = HomeHour + WorkHour + RelaxHour
WorkHour = range(8,19)      # 1
HomeHour = range(0,8)       # 2
RelaxHour = [19,20,21,22,23]     # 3

# WeekEnd = HomeWeekend + RelaxWeekend
HomeWeekend = range(0,9)    # 2
RelaxWeekend = range(9,24)  # 3

# 工作时间：1
# 居家时间：2
# 娱乐时间：3

### Option1: From Sum File

In [4]:
# From Summary File
Sumfile = '/Users/wenlanzhang/Downloads/PhD_UCL/Data/Twitter/Nairobi_1922/Sum/Summary.csv'
df = pd.read_csv(Sumfile, lineterminator='\n')
df

,id,text,userid,lat,lon,created_at
0,1079755309281890305,Quick Pose Before The Vibe @KIZAnairobi @ KIZA...,149813541,-1.290832,36.782992,2018-12-31 15:05:01+00:00
1,1079759392138907649,365/365 \n#whereveryouare #haveagoodone #beTha...,208112568,-1.283333,36.816667,2018-12-31 15:21:14+00:00
2,1079760884526972928,"Happy 2019 from us @ Nairobi, Kenya https://t....",51949523,-1.283330,36.816700,2018-12-31 15:27:10+00:00
3,1079762365644660736,"We ride together, We shine together. @ Nairobi...",89940972,-1.283330,36.816700,2018-12-31 15:33:03+00:00
4,1079764407675015170,Nairobi: 6:41pm: sunset,111780330,-1.270000,36.800000,2018-12-31 15:41:10+00:00
...,...,...,...,...,...,...
335453,1542517996685369344,find only Kenyan tea and not a blend. I have o...,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00
335454,1542517994080702464,✦ [ Sakaja ]\n\nQuestion from an American\n\nh...,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00
335455,1542518238566703108,Just posted a photo @ Adams Arcade - Ngong Roa...,1339887369495916545,-1.300567,36.780725,2022-06-30 14:39:41+00:00
335456,1542521919118770184,You better recognize....\nOwner: @kemeigabriel...,839394178824028160,-1.283330,36.816700,2022-06-30 14:54:18+00:00


In [6]:
df['created_at'] = pd.to_datetime(df['created_at'])

#  Check DayofWeek and HourOfTime
df['HourOfTime'] = df['created_at'].dt.hour
df['DayofWeek'] = df['created_at'].dt.dayofweek

#  Classifiy Time to 3 class
df['TimeClass'] = 0

for i, row in df.iterrows():
    if df.loc[i]['DayofWeek'] in Weekday:
        if df.loc[i]['HourOfTime'] in WorkHour:
            df.at[i,'TimeClass'] = 1
        elif df.loc[i]['HourOfTime'] in HomeHour:
            df.at[i,'TimeClass'] = 2
        elif df.loc[i]['HourOfTime'] in RelaxHour:
            df.at[i,'TimeClass'] = 3
        else:
            print(str(row) + str(i) + ' Wrong')

    elif df.loc[i]['DayofWeek'] in Weekend:
        if df.loc[i]['HourOfTime'] in HomeWeekend:
            df.at[i,'TimeClass'] = 2
        elif df.loc[i]['HourOfTime'] in RelaxWeekend:
            df.at[i,'TimeClass'] = 3
        else:
            print(str(row) + str(i) + ' Wrong')

df

,id,text,userid,lat,lon,created_at,HourOfTime,DayofWeek,TimeClass
0,1079755309281890305,Quick Pose Before The Vibe @KIZAnairobi @ KIZA...,149813541,-1.290832,36.782992,2018-12-31 15:05:01+00:00,15,0,3
1,1079759392138907649,365/365 \n#whereveryouare #haveagoodone #beTha...,208112568,-1.283333,36.816667,2018-12-31 15:21:14+00:00,15,0,3
2,1079760884526972928,"Happy 2019 from us @ Nairobi, Kenya https://t....",51949523,-1.283330,36.816700,2018-12-31 15:27:10+00:00,15,0,3
3,1079762365644660736,"We ride together, We shine together. @ Nairobi...",89940972,-1.283330,36.816700,2018-12-31 15:33:03+00:00,15,0,3
4,1079764407675015170,Nairobi: 6:41pm: sunset,111780330,-1.270000,36.800000,2018-12-31 15:41:10+00:00,15,0,3
...,...,...,...,...,...,...,...,...,...
335453,1542517996685369344,find only Kenyan tea and not a blend. I have o...,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00,14,3,1
335454,1542517994080702464,✦ [ Sakaja ]\n\nQuestion from an American\n\nh...,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00,14,3,1
335455,1542518238566703108,Just posted a photo @ Adams Arcade - Ngong Roa...,1339887369495916545,-1.300567,36.780725,2022-06-30 14:39:41+00:00,14,3,1
335456,1542521919118770184,You better recognize....\nOwner: @kemeigabriel...,839394178824028160,-1.283330,36.816700,2022-06-30 14:54:18+00:00,14,3,1


In [7]:
#     Dummy 3 TimeClass and Rename
df = pd.get_dummies(df, columns=['TimeClass'])
df.rename(columns={'TimeClass_1': 'Work', 'TimeClass_2': 'Home', 'TimeClass_3': 'Relax'}, inplace=True)
df

,id,text,userid,lat,lon,created_at,HourOfTime,DayofWeek,Work,Home,Relax
0,1079755309281890305,Quick Pose Before The Vibe @KIZAnairobi @ KIZA...,149813541,-1.290832,36.782992,2018-12-31 15:05:01+00:00,15,0,0,0,1
1,1079759392138907649,365/365 \n#whereveryouare #haveagoodone #beTha...,208112568,-1.283333,36.816667,2018-12-31 15:21:14+00:00,15,0,0,0,1
2,1079760884526972928,"Happy 2019 from us @ Nairobi, Kenya https://t....",51949523,-1.283330,36.816700,2018-12-31 15:27:10+00:00,15,0,0,0,1
3,1079762365644660736,"We ride together, We shine together. @ Nairobi...",89940972,-1.283330,36.816700,2018-12-31 15:33:03+00:00,15,0,0,0,1
4,1079764407675015170,Nairobi: 6:41pm: sunset,111780330,-1.270000,36.800000,2018-12-31 15:41:10+00:00,15,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
335453,1542517996685369344,find only Kenyan tea and not a blend. I have o...,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00,14,3,1,0,0
335454,1542517994080702464,✦ [ Sakaja ]\n\nQuestion from an American\n\nh...,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00,14,3,1,0,0
335455,1542518238566703108,Just posted a photo @ Adams Arcade - Ngong Roa...,1339887369495916545,-1.300567,36.780725,2022-06-30 14:39:41+00:00,14,3,1,0,0
335456,1542521919118770184,You better recognize....\nOwner: @kemeigabriel...,839394178824028160,-1.283330,36.816700,2022-06-30 14:54:18+00:00,14,3,1,0,0


In [11]:
df

,id,text,userid,lat,lon,created_at,Work,Home,Relax
0,1079755309281890305,Quick Pose Before The Vibe @KIZAnairobi @ KIZA...,149813541,-1.290832,36.782992,2018-12-31 15:05:01+00:00,0,0,1
1,1079759392138907649,365/365 \n#whereveryouare #haveagoodone #beTha...,208112568,-1.283333,36.816667,2018-12-31 15:21:14+00:00,0,0,1
2,1079760884526972928,"Happy 2019 from us @ Nairobi, Kenya https://t....",51949523,-1.283330,36.816700,2018-12-31 15:27:10+00:00,0,0,1
3,1079762365644660736,"We ride together, We shine together. @ Nairobi...",89940972,-1.283330,36.816700,2018-12-31 15:33:03+00:00,0,0,1
4,1079764407675015170,Nairobi: 6:41pm: sunset,111780330,-1.270000,36.800000,2018-12-31 15:41:10+00:00,0,0,1
...,...,...,...,...,...,...,...,...,...
335453,1542517996685369344,find only Kenyan tea and not a blend. I have o...,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00,1,0,0
335454,1542517994080702464,✦ [ Sakaja ]\n\nQuestion from an American\n\nh...,1269598733878079488,-1.292066,36.821945,2022-06-30 14:38:43+00:00,1,0,0
335455,1542518238566703108,Just posted a photo @ Adams Arcade - Ngong Roa...,1339887369495916545,-1.300567,36.780725,2022-06-30 14:39:41+00:00,1,0,0
335456,1542521919118770184,You better recognize....\nOwner: @kemeigabriel...,839394178824028160,-1.283330,36.816700,2022-06-30 14:54:18+00:00,1,0,0


In [10]:
# Export
df = df.drop(columns=['HourOfTime', 'DayofWeek'])
df.to_csv(Export + 'Time3Class', index=False)

### Option2: From Files in Folders

In [ ]:
Source = '/Users/wenlanzhang/Downloads/PhD_UCL/Data/Twitter/Nariobi/'
Destin  = '/Users/wenlanzhang/Downloads/PhD_UCL/Data/Twitter/FTR_Time/'
os.chdir(Source)

In [4]:
# From Inside folder

for file in glob.glob("*.csv"):
    print(file)
    df = pd.read_csv(file, lineterminator='\n')
#     df['created_at'] = pd.to_datetime(df['created_at'], format = '%Y%m%d %H:%M:%S')
    df['created_at'] = pd.to_datetime(df['created_at'])

#     Check DayofWeek and HourOfTime
    df['HourOfTime'] = df['created_at'].dt.hour
    df['DayofWeek'] = df['created_at'].dt.dayofweek
    
#     Classifiy Time to 3 class
    df['TimeClass'] = 0
    
    for i, row in df.iterrows():
        if df.loc[i]['DayofWeek'] in Weekday:
            if df.loc[i]['HourOfTime'] in WorkHour:
                df.at[i,'TimeClass'] = 1
            elif df.loc[i]['HourOfTime'] in HomeHour:
                df.at[i,'TimeClass'] = 2
            elif df.loc[i]['HourOfTime'] in RelaxHour:
                df.at[i,'TimeClass'] = 3
            else:
                print(str(row) + str(i) + ' Wrong')

        elif df.loc[i]['DayofWeek'] in Weekend:
            if df.loc[i]['HourOfTime'] in HomeWeekend:
                df.at[i,'TimeClass'] = 2
            elif df.loc[i]['HourOfTime'] in RelaxWeekend:
                df.at[i,'TimeClass'] = 3
            else:
                print(str(row) + str(i) + ' Wrong')
    
#     Dummy 3 TimeClass and Rename
    df = pd.get_dummies(df, columns=['TimeClass'])
    df.rename(columns={'TimeClass_1': 'Work', 'TimeClass_2': 'Home', 'TimeClass_3': 'Relax'}, inplace=True)


#     Export
    print('Done')
    df.to_csv(Export + file, index=False)


2016-03.csv
Done
2014-12.csv
Done
2022-06.csv
Done
2014-07.csv
Done
2016-02.csv
Done
2014-11.csv
Done
2022-05.csv
Done
2014-10.csv
Done
2016-01.csv
Done
2016-05.csv
Done
2016-11.csv
Done
2016-10.csv
Done
2016-04.csv
Done
2016-12.csv
Done
2016-06.csv
Done
2016-07.csv
Done
2015-08.csv
Done
2015-09.csv
Done
2015-04.csv
Done
2015-10.csv
Done
2017-01.csv
Done
2015-11.csv
Done
2015-05.csv
Done
2015-07.csv
Done
2017-02.csv
Done
2017-03.csv
Done
2015-06.csv
Done
2015-12.csv
Done
2015-02.csv
Done
2017-06.csv
Done
2015-03.csv
Done
2015-01.csv
Done
2017-04.csv
Done
2017-05.csv
Done
2016-09.csv
Done
2016-08.csv
Done
2014-09.csv
Done
2014-08.csv
Done


## Match to Grid

### DataFrame to GeoDataFrame

In [15]:
# Drop Unwanted Columns
# df = df.drop(columns=['location'])

# DataFrame to GeoDataFrame
gdfTwitter = gpd.GeoDataFrame(df, geometry = [Point(xy) for xy in zip(df.lon, df.lat)])

# Add CRS
gdfTwitter.crs = 'epsg:4326'

# type(gdfTwitter)
# gdfTwitter

### Fishnet Shp file

In [16]:
shapefilePoly = gpd.read_file('/Users/wenlanzhang/Downloads/PhD_UCL/Data/Shp/NariobiFishnet/Polygon30/NariobiFishnet.shp')

# Drop unwanted columns
shapefilePoly = shapefilePoly.drop(columns=['Shape_Area','Grid','AREA','OBJECTID','PERIMETER',
                                            'COUNTY3_', 'COUNTY3_ID','COUNTY', 'Shape_Leng'])

# Add A Column
# shapefilePoly['index'] = shapefilePoly.index   # Will generate automatically

shapefilePoly['GeometryBUP'] = shapefilePoly['geometry']

# type(shapefilePoly)
shapefilePoly

,geometry,GeometryBUP
0,"POLYGON ((36.89920 -1.16017, 36.89902 -1.16017...","POLYGON ((36.89920 -1.16017, 36.89902 -1.16017..."
1,"POLYGON ((36.89920 -1.16016, 36.89947 -1.16015...","POLYGON ((36.89920 -1.16016, 36.89947 -1.16015..."
2,"POLYGON ((36.89947 -1.16015, 36.89974 -1.16014...","POLYGON ((36.89947 -1.16015, 36.89974 -1.16014..."
3,"POLYGON ((36.89974 -1.16014, 36.90001 -1.16012...","POLYGON ((36.89974 -1.16014, 36.90001 -1.16012..."
4,"POLYGON ((36.90001 -1.16012, 36.90028 -1.16011...","POLYGON ((36.90001 -1.16012, 36.90028 -1.16011..."
...,...,...
792529,"POLYGON ((36.95903 -1.44341, 36.95924 -1.44341...","POLYGON ((36.95903 -1.44341, 36.95924 -1.44341..."
792530,"POLYGON ((36.95876 -1.44368, 36.95876 -1.44388...","POLYGON ((36.95876 -1.44368, 36.95876 -1.44388..."
792531,"POLYGON ((36.95876 -1.44368, 36.95903 -1.44368...","POLYGON ((36.95876 -1.44368, 36.95903 -1.44368..."
792532,"POLYGON ((36.95903 -1.44368, 36.95912 -1.44368...","POLYGON ((36.95903 -1.44368, 36.95912 -1.44368..."


In [17]:
shapefilePoly.count()

geometry       792534
GeometryBUP    792534
dtype: int64

### Merge/Intersect Data

In [19]:
# Intersection & Clean-up
from tqdm import tqdm

for i in tqdm(range(20), desc = 'tqdm() Progress Bar'):
    MatchedTwitter = gpd.sjoin(left_df = gdfTwitter, 
                               right_df = shapefilePoly[['geometry','GeometryBUP']], 
                               op='intersects')

MatchedTwitter
# Time: 10min

,id,text,userid,lat,lon,created_at,Work,Home,Relax,geometry,index_right,GeometryBUP
0,1079755309281890305,Quick Pose Before The Vibe @KIZAnairobi @ KIZA...,149813541,-1.290832,36.782992,2018-12-31 15:05:01+00:00,0,0,1,POINT (36.78299 -1.29083),415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
88,1079845360330293248,Smiling all the way into 2019 like....😁😁\nTo l...,254550008,-1.290832,36.782992,2018-12-31 21:02:51+00:00,0,0,1,POINT (36.78299 -1.29083),415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
174,1079946787870392320,Swizz footballers were in the building ... One...,1459793124,-1.290773,36.782847,2019-01-01 03:45:53+00:00,0,1,0,POINT (36.78285 -1.29077),415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
189,1079975641968070656,Dance !!! Party !!! Memories created @BCLUBnai...,1459793124,-1.290773,36.782847,2019-01-01 05:40:32+00:00,0,1,0,POINT (36.78285 -1.29077),415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
1018,1080937020355096577,Your view not mine ... @BCLUBnairobi \n\n#Arma...,1459793124,-1.290773,36.782847,2019-01-03 21:20:43+00:00,0,0,1,POINT (36.78285 -1.29077),415003,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061..."
...,...,...,...,...,...,...,...,...,...,...,...,...
334888,1541809039025340416,Prime video want us to get banter even before ...,207486330,-1.270000,36.720000,2022-06-28 15:41:34+00:00,1,0,0,POINT (36.72000 -1.27000),301118,"POLYGON ((36.71998 -1.26986, 36.72025 -1.26986..."
335128,1542094284584796161,Architects be architecting at the Nairobi Nati...,5989492,-1.183660,36.925449,2022-06-29 10:35:02+00:00,1,0,0,POINT (36.92545 -1.18366),13047,"POLYGON ((36.92534 -1.18362, 36.92561 -1.18362..."
335305,1542374979555180544,"Kimathi estate,bordering with Eastleigh. This ...",288236340,-1.284540,36.859900,2022-06-30 05:10:25+00:00,0,1,0,POINT (36.85990 -1.28454),383113,"POLYGON ((36.85985 -1.28441, 36.86012 -1.28441..."
335312,1542383502871625728,"I'm at Mastercard Foundation in Nairobi, Nairo...",325745714,-1.256327,36.804947,2022-06-30 05:44:17+00:00,0,1,0,POINT (36.80495 -1.25633),224592,"POLYGON ((36.80488 -1.25611, 36.80515 -1.25611..."


In [52]:
dfTextAll = MatchedTwitter[['id','Work', 'Home', 'Relax', 'GeometryBUP', 'index_right']]
dfTextAll = dfTextAll.reset_index()
dfTextAll = dfTextAll.drop(columns=['index'])
dfTextAll

,id,Work,Home,Relax,GeometryBUP,index_right
0,1079755309281890305,0,0,1,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",415003
1,1079845360330293248,0,0,1,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",415003
2,1079946787870392320,0,1,0,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",415003
3,1079975641968070656,0,1,0,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",415003
4,1080937020355096577,0,0,1,"POLYGON ((36.78278 -1.29061, 36.78305 -1.29061...",415003
...,...,...,...,...,...,...
335453,1541809039025340416,1,0,0,"POLYGON ((36.71998 -1.26986, 36.72025 -1.26986...",301118
335454,1542094284584796161,1,0,0,"POLYGON ((36.92534 -1.18362, 36.92561 -1.18362...",13047
335455,1542374979555180544,0,1,0,"POLYGON ((36.85985 -1.28441, 36.86012 -1.28441...",383113
335456,1542383502871625728,0,1,0,"POLYGON ((36.80488 -1.25611, 36.80515 -1.25611...",224592


In [74]:
dfTextAll.to_csv(Export + 'Matched.csv', index = False)

## Combine by pixcel

In [54]:
# Select repeated pixcel

Uid = list(set(dfTextAll.index_right.tolist()))
# print(Uid)
print(len(Uid))

12356


In [61]:
# Combine Time by Pixcel 

i = 0
dfnew = pd.DataFrame(columns = dfTextAll.columns)

start_time = time.monotonic()
for id in Uid:
# id = 524290
    dftemp = dfTextAll[dfTextAll['index_right'] == id]
    dftemp = dftemp.reset_index()
    dfnew.at[i,'id'] = dftemp.loc[0]['id']
    dfnew.at[i,'GeometryBUP'] = dftemp.loc[0]['GeometryBUP']
    dfnew.at[i,'index_right'] = dftemp.loc[0]['index_right']
    dfnew.at[i,'Work'] = dftemp['Work'].sum()  
    dfnew.at[i,'Home'] = dftemp['Home'].sum()  
    dfnew.at[i,'Relax'] = dftemp['Relax'].sum()  

    i = i + 1

end_time = time.monotonic()
print(timedelta(seconds = end_time - start_time))

# type(dfnew)
dfnew

0:00:12.293531


,id,Work,Home,Relax,GeometryBUP,index_right
0,1175046619101892609,1,0,0,POLYGON ((36.77577009045405 -1.292765525376451...,425988
1,1108027251159904256,1,0,3,POLYGON ((36.89865962132013 -1.314864081365799...,524293
2,1205558302668722177,6,1,4,"POLYGON ((36.8099959027786 -1.274439893580406,...",327684
3,1179109342907252737,0,1,1,POLYGON ((36.89946810507583 -1.314864081365799...,524296
4,1212070538945822720,1,0,2,POLYGON ((36.90108507258722 -1.286297655330788...,393224
...,...,...,...,...,...,...
12351,1096788749386768385,1,0,0,POLYGON ((36.72780005428265 -1.250724370079642...,196599
12352,1243222379867840513,1,0,0,"POLYGON ((36.8951561917121 -1.314864081365799,...",524280
12353,1346409324420149248,1,0,0,POLYGON ((36.77388362835742 -1.292765525376451...,425981
12354,1090155886109212672,0,1,0,POLYGON ((36.69249626361717 -1.340196572377978...,622590


### Export to Csv

In [62]:
dfnew.to_csv(Export + '3ClassDummy.csv', index = False)

### Export to Shp

In [78]:
# DataFrame to GeoDataFrame
gdfTA = gpd.GeoDataFrame(dfnew, crs="epsg:4326", geometry = dfnew['GeometryBUP'])

type(gdfTA)
# gdfTA

geopandas.geodataframe.GeoDataFrame

In [79]:
gdfTA.dtypes

id               object
Work             object
Home             object
Relax            object
GeometryBUP      object
index_right      object
geometry       geometry
dtype: object

In [80]:
gdfTA = gdfTA.drop(columns=['GeometryBUP', 'index_right'])
gdfTA.dtypes
gdfTA.head(50)

,id,Work,Home,Relax,geometry
0,1175046619101892609,1,0,0,"POLYGON ((36.77577 -1.29277, 36.77604 -1.29277..."
1,1108027251159904256,1,0,3,"POLYGON ((36.89866 -1.31486, 36.89893 -1.31486..."
2,1205558302668722177,6,1,4,"POLYGON ((36.81000 -1.27444, 36.81027 -1.27444..."
3,1179109342907252737,0,1,1,"POLYGON ((36.89947 -1.31486, 36.89974 -1.31486..."
4,1212070538945822720,1,0,2,"POLYGON ((36.90109 -1.28630, 36.90135 -1.28630..."
5,1450213373568823300,0,0,2,"POLYGON ((36.90162 -1.28630, 36.90189 -1.28630..."
6,1134033900039069696,1,0,0,"POLYGON ((36.81215 -1.27444, 36.81242 -1.27444..."
7,1104989467591524353,0,1,0,"POLYGON ((36.73858 -1.29977, 36.73885 -1.29977..."
8,1494350798276419595,1,1,0,"POLYGON ((36.81296 -1.27444, 36.81323 -1.27444..."
9,1084360887543255040,4,4,1,"POLYGON ((36.77900 -1.29277, 36.77927 -1.29277..."


In [81]:
gdfTA['id'] = gdfTA['id'].astype("int")
gdfTA['Work'] = gdfTA['Work'].astype("int")
gdfTA['Home'] = gdfTA['Home'].astype("int")
gdfTA['Relax'] = gdfTA['Relax'].astype("int")

gdfTA.dtypes

id             int64
Work           int64
Home           int64
Relax          int64
geometry    geometry
dtype: object

In [82]:
gdfTA

,id,Work,Home,Relax,geometry
0,1175046619101892609,1,0,0,"POLYGON ((36.77577 -1.29277, 36.77604 -1.29277..."
1,1108027251159904256,1,0,3,"POLYGON ((36.89866 -1.31486, 36.89893 -1.31486..."
2,1205558302668722177,6,1,4,"POLYGON ((36.81000 -1.27444, 36.81027 -1.27444..."
3,1179109342907252737,0,1,1,"POLYGON ((36.89947 -1.31486, 36.89974 -1.31486..."
4,1212070538945822720,1,0,2,"POLYGON ((36.90109 -1.28630, 36.90135 -1.28630..."
...,...,...,...,...,...
12351,1096788749386768385,1,0,0,"POLYGON ((36.72780 -1.25072, 36.72807 -1.25072..."
12352,1243222379867840513,1,0,0,"POLYGON ((36.89516 -1.31486, 36.89543 -1.31486..."
12353,1346409324420149248,1,0,0,"POLYGON ((36.77388 -1.29277, 36.77415 -1.29277..."
12354,1090155886109212672,0,1,0,"POLYGON ((36.69250 -1.34020, 36.69277 -1.34020..."


In [83]:
gdfTA.to_file(Export + 'Grided.shp', index = False) 

Test

In [7]:
file = '2022-05.csv'
df = pd.read_csv(Source + file)
df

,id,text,userid,lat,lon,created_at,location
0,1522428904555917314,Vans shoes \nDifferent colours\nSizes 37-44\nR...,389054956,-1.283330,36.816700,Fri May 06 04:11:50 +0000 2022,Nairobi
1,1522435815724527617,Just turned 2. \nGod is always amazing\n.\n.\n...,769651362569281538,-1.317000,36.825000,Fri May 06 04:39:18 +0000 2022,Nairobi
2,1522436896672161792,✦ [ #MaishaConcertFriday ]\n\nKARMA TRAIN\n\nh...,1269598733878079488,-1.292066,36.821945,Fri May 06 04:43:36 +0000 2022,Nairobi
3,1522436897712349186,"Hi everyone, so many posts here, responses but...",1269598733878079488,-1.292066,36.821945,Fri May 06 04:43:36 +0000 2022,Nairobi
4,1522437504057573377,JORDAN\nSIZES 37-44\nRETAIL @2500\nCALL +25472...,389054956,-1.283330,36.816700,Fri May 06 04:46:01 +0000 2022,Nairobi
...,...,...,...,...,...,...,...
7784,1520467584965128194,"adding VAT tax to flour. As you might know, fl...",1269598733878079488,-1.292066,36.821945,Sat Apr 30 18:18:15 +0000 2022,Nairobi
7785,1520467586051461122,a national crisis? IDK. The 5th is the dollar ...,1269598733878079488,-1.292066,36.821945,Sat Apr 30 18:18:16 +0000 2022,Nairobi
7786,1520467587129352193,net importer of maize. Will this affect produc...,1269598733878079488,-1.292066,36.821945,Sat Apr 30 18:18:16 +0000 2022,Nairobi
7787,1520472105543942146,✦ [ #HalaMadrid ] ● Ask r/kenya\n\nHow many ye...,1269598733878079488,-1.292066,36.821945,Sat Apr 30 18:36:13 +0000 2022,Nairobi


In [8]:
df['created_at'] = pd.to_datetime(df['created_at'])
df

,id,text,userid,lat,lon,created_at,location
0,1522428904555917314,Vans shoes \nDifferent colours\nSizes 37-44\nR...,389054956,-1.283330,36.816700,2022-05-06 04:11:50+00:00,Nairobi
1,1522435815724527617,Just turned 2. \nGod is always amazing\n.\n.\n...,769651362569281538,-1.317000,36.825000,2022-05-06 04:39:18+00:00,Nairobi
2,1522436896672161792,✦ [ #MaishaConcertFriday ]\n\nKARMA TRAIN\n\nh...,1269598733878079488,-1.292066,36.821945,2022-05-06 04:43:36+00:00,Nairobi
3,1522436897712349186,"Hi everyone, so many posts here, responses but...",1269598733878079488,-1.292066,36.821945,2022-05-06 04:43:36+00:00,Nairobi
4,1522437504057573377,JORDAN\nSIZES 37-44\nRETAIL @2500\nCALL +25472...,389054956,-1.283330,36.816700,2022-05-06 04:46:01+00:00,Nairobi
...,...,...,...,...,...,...,...
7784,1520467584965128194,"adding VAT tax to flour. As you might know, fl...",1269598733878079488,-1.292066,36.821945,2022-04-30 18:18:15+00:00,Nairobi
7785,1520467586051461122,a national crisis? IDK. The 5th is the dollar ...,1269598733878079488,-1.292066,36.821945,2022-04-30 18:18:16+00:00,Nairobi
7786,1520467587129352193,net importer of maize. Will this affect produc...,1269598733878079488,-1.292066,36.821945,2022-04-30 18:18:16+00:00,Nairobi
7787,1520472105543942146,✦ [ #HalaMadrid ] ● Ask r/kenya\n\nHow many ye...,1269598733878079488,-1.292066,36.821945,2022-04-30 18:36:13+00:00,Nairobi


In [9]:
df['HourOfTime'] = df['created_at'].dt.hour
df['DayofWeek'] = df['created_at'].dt.dayofweek
df

,id,text,userid,lat,lon,created_at,location,HourOfTime,DayofWeek
0,1522428904555917314,Vans shoes \nDifferent colours\nSizes 37-44\nR...,389054956,-1.283330,36.816700,2022-05-06 04:11:50+00:00,Nairobi,4,4
1,1522435815724527617,Just turned 2. \nGod is always amazing\n.\n.\n...,769651362569281538,-1.317000,36.825000,2022-05-06 04:39:18+00:00,Nairobi,4,4
2,1522436896672161792,✦ [ #MaishaConcertFriday ]\n\nKARMA TRAIN\n\nh...,1269598733878079488,-1.292066,36.821945,2022-05-06 04:43:36+00:00,Nairobi,4,4
3,1522436897712349186,"Hi everyone, so many posts here, responses but...",1269598733878079488,-1.292066,36.821945,2022-05-06 04:43:36+00:00,Nairobi,4,4
4,1522437504057573377,JORDAN\nSIZES 37-44\nRETAIL @2500\nCALL +25472...,389054956,-1.283330,36.816700,2022-05-06 04:46:01+00:00,Nairobi,4,4
...,...,...,...,...,...,...,...,...,...
7784,1520467584965128194,"adding VAT tax to flour. As you might know, fl...",1269598733878079488,-1.292066,36.821945,2022-04-30 18:18:15+00:00,Nairobi,18,5
7785,1520467586051461122,a national crisis? IDK. The 5th is the dollar ...,1269598733878079488,-1.292066,36.821945,2022-04-30 18:18:16+00:00,Nairobi,18,5
7786,1520467587129352193,net importer of maize. Will this affect produc...,1269598733878079488,-1.292066,36.821945,2022-04-30 18:18:16+00:00,Nairobi,18,5
7787,1520472105543942146,✦ [ #HalaMadrid ] ● Ask r/kenya\n\nHow many ye...,1269598733878079488,-1.292066,36.821945,2022-04-30 18:36:13+00:00,Nairobi,18,5


In [10]:
# Check Date of the week print out
WeekIndex = list(set(df.DayofWeek.tolist()))
# len(UniqueIndex)
WeekIndex

[0, 1, 2, 3, 4, 5, 6]

In [11]:
# Check Hour print out
HourIndex = list(set(df.HourOfTime.tolist()))
# len(UniqueIndex)
HourIndex

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]

In [12]:
WorkHour = range(8,18)
print(WorkHour)

range(8, 18)


In [13]:
Weekday = [1, 2, 3, 4, 5]
Weekend = [0,6]

# WeekDay = HomeHour + WorkHour + RelaxHour
WorkHour = range(8,19)      # 1
HomeHour = range(0,8)       # 2
RelaxHour = [19,20,21,22,23]     # 3

# WeekEnd = HomeWeekend + RelaxWeekend
HomeWeekend = range(0,9)    # 2
RelaxWeekend = range(9,24)  # 3

# 工作时间：1
# 居家时间：2
# 娱乐时间：3

df['TimeClass'] = 0

# print(all([isinstance(item, int) for item in Weekday]))
for i, row in df.iterrows():
    if df.loc[i]['DayofWeek'] in Weekday:
        if df.loc[i]['HourOfTime'] in WorkHour:
            df.at[i,'TimeClass'] = 1
        elif df.loc[i]['HourOfTime'] in HomeHour:
            df.at[i,'TimeClass'] = 2
        elif df.loc[i]['HourOfTime'] in RelaxHour:
            df.at[i,'TimeClass'] = 3
        else:
            print(str(row) + str(i) + ' Wrong')
                
    elif df.loc[i]['DayofWeek'] in Weekend:
        if df.loc[i]['HourOfTime'] in HomeWeekend:
            df.at[i,'TimeClass'] = 2
        elif df.loc[i]['HourOfTime'] in RelaxWeekend:
            df.at[i,'TimeClass'] = 3
        else:
            print(str(row) + str(i) + ' Wrong')

In [34]:
# Check Date of the week print out
TimeClassIndex = list(set(df.TimeClass.tolist()))
TimeClassIndex

[1, 2, 3]

In [14]:
df = pd.get_dummies(df, columns=['TimeClass'])
df

,id,text,userid,lat,lon,created_at,location,HourOfTime,DayofWeek,TimeClass_1,TimeClass_2,TimeClass_3
0,1522428904555917314,Vans shoes \nDifferent colours\nSizes 37-44\nR...,389054956,-1.283330,36.816700,2022-05-06 04:11:50+00:00,Nairobi,4,4,0,1,0
1,1522435815724527617,Just turned 2. \nGod is always amazing\n.\n.\n...,769651362569281538,-1.317000,36.825000,2022-05-06 04:39:18+00:00,Nairobi,4,4,0,1,0
2,1522436896672161792,✦ [ #MaishaConcertFriday ]\n\nKARMA TRAIN\n\nh...,1269598733878079488,-1.292066,36.821945,2022-05-06 04:43:36+00:00,Nairobi,4,4,0,1,0
3,1522436897712349186,"Hi everyone, so many posts here, responses but...",1269598733878079488,-1.292066,36.821945,2022-05-06 04:43:36+00:00,Nairobi,4,4,0,1,0
4,1522437504057573377,JORDAN\nSIZES 37-44\nRETAIL @2500\nCALL +25472...,389054956,-1.283330,36.816700,2022-05-06 04:46:01+00:00,Nairobi,4,4,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7784,1520467584965128194,"adding VAT tax to flour. As you might know, fl...",1269598733878079488,-1.292066,36.821945,2022-04-30 18:18:15+00:00,Nairobi,18,5,1,0,0
7785,1520467586051461122,a national crisis? IDK. The 5th is the dollar ...,1269598733878079488,-1.292066,36.821945,2022-04-30 18:18:16+00:00,Nairobi,18,5,1,0,0
7786,1520467587129352193,net importer of maize. Will this affect produc...,1269598733878079488,-1.292066,36.821945,2022-04-30 18:18:16+00:00,Nairobi,18,5,1,0,0
7787,1520472105543942146,✦ [ #HalaMadrid ] ● Ask r/kenya\n\nHow many ye...,1269598733878079488,-1.292066,36.821945,2022-04-30 18:36:13+00:00,Nairobi,18,5,1,0,0


In [16]:
df.rename(columns={'TimeClass_1': 'Work', 'TimeClass_2': 'Home', 'TimeClass_3': 'Relax'}, inplace=True)
df

,id,text,userid,lat,lon,created_at,location,HourOfTime,DayofWeek,Work,HomeHour,RelaxHour
0,1522428904555917314,Vans shoes \nDifferent colours\nSizes 37-44\nR...,389054956,-1.283330,36.816700,2022-05-06 04:11:50+00:00,Nairobi,4,4,0,1,0
1,1522435815724527617,Just turned 2. \nGod is always amazing\n.\n.\n...,769651362569281538,-1.317000,36.825000,2022-05-06 04:39:18+00:00,Nairobi,4,4,0,1,0
2,1522436896672161792,✦ [ #MaishaConcertFriday ]\n\nKARMA TRAIN\n\nh...,1269598733878079488,-1.292066,36.821945,2022-05-06 04:43:36+00:00,Nairobi,4,4,0,1,0
3,1522436897712349186,"Hi everyone, so many posts here, responses but...",1269598733878079488,-1.292066,36.821945,2022-05-06 04:43:36+00:00,Nairobi,4,4,0,1,0
4,1522437504057573377,JORDAN\nSIZES 37-44\nRETAIL @2500\nCALL +25472...,389054956,-1.283330,36.816700,2022-05-06 04:46:01+00:00,Nairobi,4,4,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7784,1520467584965128194,"adding VAT tax to flour. As you might know, fl...",1269598733878079488,-1.292066,36.821945,2022-04-30 18:18:15+00:00,Nairobi,18,5,1,0,0
7785,1520467586051461122,a national crisis? IDK. The 5th is the dollar ...,1269598733878079488,-1.292066,36.821945,2022-04-30 18:18:16+00:00,Nairobi,18,5,1,0,0
7786,1520467587129352193,net importer of maize. Will this affect produc...,1269598733878079488,-1.292066,36.821945,2022-04-30 18:18:16+00:00,Nairobi,18,5,1,0,0
7787,1520472105543942146,✦ [ #HalaMadrid ] ● Ask r/kenya\n\nHow many ye...,1269598733878079488,-1.292066,36.821945,2022-04-30 18:36:13+00:00,Nairobi,18,5,1,0,0
